In [ ]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import glob # A module to match regular expressions in this case we want to match all csv files in a folder and flush them out!
import calendar
from tqdm import tqdm
plt.rcParams["figure.figsize"] = (18, 8)
legend_size = 15
plt.rcParams['legend.title_fontsize'] = legend_size
plt.rcParams['xtick.labelsize']=20
plt.rcParams['ytick.labelsize']=20
plt.rcParams['axes.labelsize']=20
plt.rcParams['axes.titlesize']=20

# **Section 1: Data Preprocessing**

In [ ]:
data_path = "data"

In [ ]:
# Lets get all files with their respective paths 
all_csv_files = glob.glob(f'{data_path}/*y.csv')
all_csv_files

In [ ]:
# Create a function that orders day of week
def order_files_by_day_of_week(files):
    filenames = []
    days = [i.split("/")[1].split(".")[0].title() for i in all_csv_files]
    for ordered_day in list(calendar.day_name):
        if ordered_day in days:
            index = days.index(ordered_day)
            filenames.append(files[index])
    return filenames

In [ ]:
# Get ordered files 
files = order_files_by_day_of_week(all_csv_files)
files

In [ ]:
# Read the files
df_list = [pd.read_csv(i, index_col = 0, parse_dates=True, sep=";") for i in files]

In [ ]:
# Get abbreviated weekly names from files 
weekly_names = [i.split("/")[1].split(".")[0][0:3] for i in files]
weekly_names

In [ ]:
# Put dataframe list into a dictionary 
df_dict = {}
count = 0 
for week_name in weekly_names:
    df_dict[week_name] = df_list[count]
    count += 1
calendar.day_name[calendar.firstweekday()]

In [ ]:
# Give unique id to a customer and concatenate into one huge dataframe

df_all = [] 
for key in df_dict.keys():
    df = df_dict[key]
    df["day_of_week"] = df.index.day_name()
    df["customer_no"] = df["customer_no"].astype(str)
    df["shortened_day"] = [i[0:3] for i in df["day_of_week"].tolist()]
    df["customer_id"] = df["customer_no"] + "_" + df["shortened_day"]
    df.drop("shortened_day", axis = 1, inplace=True)
    df_all.append(df)

df_all = pd.concat(df_all)

In [ ]:
# df_mo = pd.read_csv(f'{data_path}/monday.csv', index_col = 0, parse_dates=True, sep=';')
# df_tu = pd.read_csv(f'{data_path}/tuesday.csv',  index_col = 0,parse_dates=True, sep=';')
# df_we = pd.read_csv(f'{data_path}/wednesday.csv',  index_col = 0,parse_dates=True, sep=';')
# df_th = pd.read_csv(f'{data_path}/thursday.csv',  index_col = 0,parse_dates=True, sep=';')
# df_fr = pd.read_csv(f'{data_path}/friday.csv',  index_col = 0,parse_dates=True, sep=';')


In [ ]:
# df_all = pd.concat([df_mo, df_tu, df_we, df_th, df_fr])

In [ ]:
df_all.shape

In [ ]:
# Add hour and minute column
# df_all['date'] = pd.to_datetime(df_all['timestamp'])
df_all["hour"] = df_all.index.hour
df_all["minute"] = df_all.index.minute

In [ ]:
# df_all['day'] =df_all['date'].dt.day_name()
# df_all['hour'] =df_all['date'].dt.hour
# df_all.head()

In [ ]:
df_all['location'].value_counts()

# **Section 2: Real Deal (Answering the questions)**

# 1. Calculate the total number of customers in each section

In [ ]:
sns.countplot(x=df_all.location)

# 2a. Calculate the total number of customers in each section over time (day)

In [ ]:
sns.countplot(x=df_all.location, hue=df_all.day_of_week)
plt.legend(bbox_to_anchor=(1.1, 0.5), loc=5, title = "day of week", fontsize = 15)

# 2b. Calculate the total number of customers in each section over time: hour

In [ ]:
sns.countplot(x=df_all.location, hue=df_all.hour)
plt.legend(bbox_to_anchor=(1.17, 0.5), loc=5, title="Hour of day", prop = {"size": legend_size})
plt.title("Hourly customer count per section", fontsize = 30, fontweight = "bold")

df_all[["customer_id"]].groupby(df_all["location"]).agg("count")

def checkout_by_hour(df):
    # Get all hour timesteps 
    hours = [i for i in df.index.hour.unique()]
    df_list = []
    for hour in hours: 
        df_hour = df[df.index.hour == hour]
        df_list.append(pd.DataFrame(df_hour["customer_id"].groupby(df_hour["location"]).count()))
    final_df = pd.concat(df_list, axis = 1)
    final_df.columns = hours 
    return final_df

df_hour_location = checkout_by_hour(df_all)

#sns.heatmap(my_df)
df_hour_location.plot(kind = "bar")
plt.ylabel("Count")
plt.legend(prop={"size": legend_size}, title = "Hour of day")
df_hour_location

# 2c. Calculate the total number of customers in each section over time: minute

In [ ]:

sns.countplot(hue=df_all.location, x=df_all.minute)
plt.legend(bbox_to_anchor=(1.1, 0.5), loc=5, title = "Location", fontsize = 15)
plt.title("Minute customer count per section", fontsize = 30, fontweight = "bold")

# 3. Display the number of customers at checkout over time

In [ ]:
df_checkout = df_all[df_all["location"] == "checkout"]
#sns.countplot(x=df_checkout["location"], hue = df_checkout.index.hour)
df_checkout["location"].groupby(df_checkout.index.hour).count().plot(kind = "bar")
plt.title("Customer checkout per hour", fontsize = 20)


# 4. Calculate the time each customer spent in the market


In [ ]:
customer_ids = [i for i in df_all["customer_id"].unique()]

duration = []
time_in = []
time_out = []

for customer_id in customer_ids:

    df_day = df_all[df_all["customer_id"] == customer_id]

    #print(df_day)
    if "checkout" in df_day["location"].tolist():
        min_time = df_day.index.min()
        max_time = df_day.index.max()
        diff = (max_time - min_time).seconds
        duration.append(diff)
        time_in.append(str(min_time))
        time_out.append(str(max_time))
   
    else:
        min_time = df_day.index.min()
        construct_last_time = datetime.strptime(f'{str(max_time).split(" ")[0]} 21:59:59', "%Y-%m-%d %H:%M:%S")
        diff = construct_last_time - min_time
        diff = diff.seconds
        duration.append(diff)  
        time_in.append(str(min_time))
        time_out.append(str(construct_last_time))
      

## Create a new dataframe with customer id, time in, time out and duration inside supermarket

In [ ]:
df_duration = pd.DataFrame({"customer_id": customer_ids, "entry time": time_in, "exit time": time_out, "duration (secs)": duration})
df_duration["duration (mins)"] = (df_duration["duration (secs)"]/60).round(2)
df_duration



# 5. Calculate the total number of customers in the supermarket over time
## We calculate per hour

In [ ]:
df_customers_per_hour = pd.DataFrame(df_hour_location.sum(axis = 0), columns = ["total"])
df_customers_per_hour["hour"] = df_customers_per_hour.index
df_customers_per_hour["total"].plot(kind = "bar")
plt.title("No of customers in the market")
plt.ylabel("Count")
plt.xlabel("Hour of day")

In [ ]:
def convert_digit_to_weekday(x):
    if x == 0:
        return "Monday"
    if x == 1:
        return "Tuesday"
    if x == 2:
        return "Wednesday"
    if x == 3:
        return "Thursday"
    if x == 4:
        return "Friday"
    if x == 5:
        return "Saturday"
    if x == 6:
        return "Sunday"
        
## We calculate per day 
df_day_total = pd.DataFrame(df_all["customer_id"].groupby(df_all.index.dayofweek).count())
df_day_total["day"] = [convert_digit_to_weekday(i) for i in df_day_total.index]
df_day_total.columns = ["total", "day"]
df_day_total[["day", "total"]].plot(kind = "bar")
x = df_day_total.index.tolist()
labels = df_day_total["day"].tolist()
plt.xticks(x, labels, rotation = 45)
plt.xlabel("")

# 5. Distribution of customers of their first visited section versus following sections

In [ ]:
import warnings
warnings.filterwarnings('ignore')
customer_ids = [i for i in df_all["customer_id"].unique()]

first_visited = []
following = []
df_list = []

for customer_id in tqdm(customer_ids):
    #print(f'{customer_id}/{len(customer_ids)}')
    df_per_customer = df_all[df_all["customer_id"] == customer_id]
    df_per_customer["location_pattern"] = None
    df_per_customer["location_pattern"][0] = "first visited"
    df_per_customer["location_pattern"][1:] = "following"
    df_list.append(df_per_customer)
df_visit = pd.concat(df_list, axis = 0)

df_visit
        